In [1]:
import pandas as pd 
import numpy as np 
from google.colab import drive 

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
datos_2022=pd.read_csv('/content/drive/MyDrive/datos_febrero_2022_.csv').drop(columns='Unnamed: 0')
datos_2022.head(3)

,entidad,viv_sel,num_hog,hog_mud,n_ren_el,cd,sexo,edad,nivel,cond_act,...,p36_2,p36_3,p36_4,h_lec,mat_lec,perslec,l_format,r_format,p_format,perslecl
0,1,4,1,0,1,14,1,51,6,1,...,1,1,2,3,3,1,0,0,0,2
1,1,2,1,0,2,14,2,73,2,7,...,1,1,2,4,4,2,0,0,0,2
2,1,1,1,0,1,14,1,52,3,1,...,1,1,2,3,3,1,0,0,0,2


In [4]:
diccionario=pd.read_csv('/content/drive/MyDrive/diccionario_b.csv').drop(columns='Unnamed: 0')
diccionario.head(110)

,NOMBRE_CAMPO,Claves,Catálogo
0,Folio,folio,NaN
1,Entidad,entidad,entidad
2,Control,control,NaN
3,Vivienda seleccionada,viv_sel,viv_sel
4,Número de hogar,num_hog,num_hog
...,...,...,...
103,Status de las personas en condición de lectura...,perslec,cpperslec
104,Formato de libros,l_format,cp_formato
105,Formato de revistas,r_format,cp_formato
106,Formato de periódicos,p_format,cp_formato


El MOLEC (Modulo sobre Lectura) del INEGI considera las lecturas de libros, revistas, periódicos, historietas y páginas de Internet, foros o blogs

#¿En que estado se lee más?

In [5]:
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [6]:
datos_lectores=datos_2022[datos_2022['perslecl']==1]

lectura_por_estado= pd.DataFrame(datos_lectores.groupby('entidad').perslecl.sum()).rename(columns={'perslecl': 'num_lectores'}).reset_index()
lectura_por_estado.head(3)

,entidad,num_lectores
0,1,18
1,2,26
2,3,25


In [7]:
encuestas_por_estado=pd.DataFrame(datos_2022.groupby('entidad').entidad.value_counts())
encuestas_por_estado= encuestas_por_estado.rename(columns={'entidad': 'encuestados por entidad'}).reset_index(level=0)
encuestas_por_estado.drop(columns='entidad', inplace=True)
encuestas_por_estado=encuestas_por_estado.reset_index()
encuestas_por_estado.head(3)

,entidad,encuestados por entidad
0,1,47
1,2,40
2,3,34


In [ ]:
encuestas_por_estado['encuestados por entidad'].sum()

2016

In [8]:
lectura_por_estado['encuestados por entidad']=encuestas_por_estado['encuestados por entidad']
lectura_por_estado['porcentaje de lectores']=100*lectura_por_estado['num_lectores']/encuestas_por_estado['encuestados por entidad']
lectura_por_estado.head(3)

,entidad,num_lectores,encuestados por entidad,porcentaje de lectores
0,1,18,47,38.297872
1,2,26,40,65.000000
2,3,25,34,73.529412


In [31]:
trace = go.Bar(
                    x = lectura_por_estado['entidad'],
                    y = lectura_por_estado['porcentaje de lectores'] ,
                    orientation='v',
                    marker = dict(color='pink',
                                 line=dict(color='black',width=1)),
                    )
data = [trace]
layout = go.Layout(barmode = "group",title="Porcentaje de lectores por estado registrado en la encuesta 2022",width=800, height=500, 
                       xaxis= dict(title='Estados'),
                       yaxis=dict(title="% Lectores"))

In [38]:
fig0= go.Figure(data = data, layout = layout)
iplot(fig0)

#Información sobre posible factor para determinar si es un lector o no

In [ ]:
datos_2022['perslecl'].unique()

array([2, 1, 0])

In [11]:
datos_infancia_1=datos_2022[datos_2022['perslecl']==1].groupby('p34_3')['perslecl'].value_counts()
datos_infancia_1=pd.DataFrame(datos_infancia_1)
datos_infancia_1.rename(columns={'perslecl':'num_personas'},inplace=True)
datos_infancia_1=datos_infancia_1.reset_index()
datos_infancia_1.rename(columns={'p34_3':'padres_lectores'},inplace=True)

In [12]:
datos_infancia_1=datos_infancia_1.drop(columns='perslecl')
datos_infancia_1

,padres_lectores,num_personas
0,1,545
1,2,646
2,3,32


In [13]:
datos_infancia_2=datos_2022[datos_2022['perslecl']==2].groupby('p34_3')['perslecl'].value_counts()
datos_infancia_2=pd.DataFrame(datos_infancia_2)
datos_infancia_2.rename(columns={'perslecl':'num_personas'},inplace=True)
datos_infancia_2=datos_infancia_2.reset_index()
datos_infancia_2.rename(columns={'p34_3':'padres_lectores'},inplace=True)

In [14]:
datos_infancia_2=datos_infancia_2.drop(columns='perslecl')
datos_infancia_2

,padres_lectores,num_personas
0,1,163
1,2,558
2,3,35


In [19]:
trace = go.Pie(
                    labels = ['Sí me leía', 'No me leía', 'No recuerdo'],
                    values= datos_infancia_1['num_personas']
                    )
data = [trace]
layout = go.Layout(title="Lectura de los padres de las personas lectoras",width=600, height=500 )
trace1 = go.Pie(
                    labels = ['Sí me leía', 'No me leía', 'No recuerdo'],
                    values = datos_infancia_2['num_personas']
                    )
data2 = [trace1]
layout2 = go.Layout(title="Lectura de los padres de las personas no lectoras",width=600, height=500)

In [20]:
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [21]:
fig2= go.Figure(data = data2, layout = layout2)
iplot(fig2)

#Lectores por grados escolares

In [22]:
num_encuestados_por_grados=pd.DataFrame(datos_2022.groupby('nivel').nivel.value_counts())
num_encuestados_por_grados=num_encuestados_por_grados.rename(columns={'nivel':'num_personas'}).reset_index(level=0)
num_encuestados_por_grados.drop(columns='nivel', inplace=True)
num_encuestados_por_grados=num_encuestados_por_grados.reset_index()
num_encuestados_por_grados.head(3)

,nivel,num_personas
0,0,39
1,2,342
2,3,473


In [23]:
lectores_por_grado= datos_lectores.groupby('nivel').perslec.value_counts()
lectores_por_grado=pd.DataFrame(lectores_por_grado).rename(columns={'perslec': 'num_lectores'})
lectores_por_grado=lectores_por_grado.reset_index()
lectores_por_grado.drop(columns='perslec', inplace=True)
lectores_por_grado.head(3)

,nivel,num_lectores
0,0,7
1,2,143
2,3,246


In [24]:
lectores_por_grado['num_entrevistados']=num_encuestados_por_grados['num_personas']
lectores_por_grado['% de lectores por grado']=100*lectores_por_grado['num_lectores']/num_encuestados_por_grados['num_personas']
lectores_por_grado.head(3)

,nivel,num_lectores,num_entrevistados,% de lectores por grado
0,0,7,39,17.948718
1,2,143,342,41.812865
2,3,246,473,52.008457


In [25]:
trace=go.Bar(
    x=['ninguno','preescolar','primaria', 'secundaria', 'Preparatoria', 'normal basica', 'carrera tecnica', 'profesional', 'maestría', 'doctorado'],
    y=lectores_por_grado['% de lectores por grado'],
    orientation='v',
    marker = dict(color='pink', line=dict(color='black',width=1)),
)
data=[trace]

layout= go.Layout(barmode = "group",title="Lectores por grado",width=800, height=500, 
                       xaxis= dict(title='grados'),
                       yaxis=dict(title="porcentaje de lectores"))

In [26]:
fig=go.Figure(data=data,layout=layout)
iplot(fig)

#Otro datos más
Existe un campo llamado vivienda seleccionada, si nos vamos al catálogo solo menciona si es una viivenda del tipo 1, del tipo 2, del tipo 3 o del tipo 4. No se añade más información sobre el campo, queda pendiente investigar que representan estas clasificaciones de viviendas, pero se analiza si existe algún tipo de relación entre el tipo de vivienda con la cantidad de lectores

In [ ]:
num_encuestados_por_viv=pd.DataFrame(datos_2022.groupby('viv_sel').viv_sel.value_counts())
num_encuestados_por_viv=num_encuestados_por_viv.rename(columns={'viv_sel':'num_personas'}).reset_index(level=0)
num_encuestados_por_viv.drop(columns='viv_sel', inplace=True)
num_encuestados_por_viv=num_encuestados_por_viv.reset_index()
num_encuestados_por_viv.head(4)

,viv_sel,num_personas
0,1,516
1,2,520
2,3,487
3,4,493


In [ ]:
lectores_por_viv= datos_lectores.groupby('viv_sel').perslecl.value_counts()
lectores_por_viv=pd.DataFrame(lectores_por_viv).rename(columns={'perslecl': 'num_lectores'})
lectores_por_viv=lectores_por_viv.reset_index()
lectores_por_viv.drop(columns='perslecl', inplace=True)
lectores_por_viv.head(4)

,viv_sel,num_lectores
0,1,306
1,2,327
2,3,303
3,4,287


In [ ]:
lectores_por_viv['num_encuestados']=num_encuestados_por_viv['num_personas']
lectores_por_viv['% de lectores por viv']=100*lectores_por_viv['num_lectores']/num_encuestados_por_viv['num_personas']
lectores_por_viv

,viv_sel,num_lectores,num_encuestados,% de lectores por viv
0,1,306,516,59.302326
1,2,327,520,62.884615
2,3,303,487,62.217659
3,4,287,493,58.215010


Parece que se distribuyen los lectores de una manera medianamente uniforme, la caracteristica vivienda selecionada no parece ser una caracteristica determinante para saber si es lector o no 

In [33]:
!pip install dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 3.3 MB/s 
     |████████████████████████████████| 357 kB 52.6 MB/s 


In [35]:
import dash
import dash_core_components as dcc
import dash_html_components as html 
from dash.dependencies import Input, Output

In [41]:
app=dash.Dash(__name__)
app.layout=html.Div([
                     
                     html.H1('ALgunas estadisticas del Modulo de Lectura 2022 INEGI', style={'text-align': 'center'}),

                     dcc.Dropdown(id='selec', 
                                  options=[
                                           {'label': 'Lectura por estado', 'value':'lectura_por_estado'},
                                           {'label': 'lectura por grado', 'value': 'lectores_por_grado'}],
                                           multi=False,
                                           value ='lectores_por_grado',
                                          style={'with': '40%'}
                                  ), 
                     dcc.Graph(id='my_bar', figure={})
])
@app.callback(
    [Output(component_id='my_bar', component_property='figure')],
    [Input(component_id='selec', component_property='value')]

)
def actualizar_grafico(opccion):
  if opccion=='lectura_por_estado':
    return fig0
  elif opccion=='lectores_por_grado':
    return fig